In [679]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
import os
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB





In [680]:
bd = load_files('baise_spamfilter')


In [681]:
bd.keys()

dict_keys(['DESCR', 'target', 'filenames', 'data', 'target_names'])

In [682]:
len(bd.data)

14

In [683]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [684]:
#ENGLISH_STOP_WORDS

In [685]:
print(bd.target_names)

['ham', 'spam']


In [686]:
print(bd.target_names[bd.target[1]],"\n\n",bd.data[1])

ham 

 b'\xef\xbb\xbfThis is just a reminder that today from\xc2\xa05:30 to 7:30\xc2\xa0there will be community members and their children walking through our halls trick or treating.\xc2\xa0\n\nThere will be signs indicating that rooms with pumpkins on them are handing out candy. If you did not sign up to hand out candy but would like to get involved there will be cookie decorating in the second floor lounge.\xc2\xa0\n\nLets make sure that the community members leave our hall with not only candy but a great experience as well.\xc2\xa0\n\nAl\n\n'


In [687]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=False)

In [688]:
X = vectorizer.fit_transform(bd.data)                            
#X #sparse matrix
#len(vectorizer.get_feature_names()) #549

In [689]:

for i in range(14):
    for j in range(1082):
        if X[i,j] != 0.0:
            print("document = ", i, "score = ",j,X[i,j], vectorizer.get_feature_names()[j])

document =  0 score =  18 0.0426014322842 25
document =  0 score =  21 0.0426014322842 260
document =  0 score =  26 0.0426014322842 30
document =  0 score =  38 0.0426014322842 60
document =  0 score =  46 0.0426014322842 95
document =  0 score =  70 0.0426014322842 an
document =  0 score =  71 0.255608593705 and
document =  0 score =  76 0.0852028645685 are
document =  0 score =  78 0.0426014322842 as
document =  0 score =  91 0.0426014322842 because
document =  0 score =  93 0.0852028645685 before
document =  0 score =  95 0.0426014322842 behavior
document =  0 score =  99 0.0426014322842 better
document =  0 score =  101 0.0426014322842 big
document =  0 score =  102 0.0426014322842 billions
document =  0 score =  104 0.0426014322842 bombs
document =  0 score =  110 0.0426014322842 buy
document =  0 score =  152 0.0426014322842 comfort
document =  0 score =  154 0.170405729137 companies
document =  0 score =  163 0.0426014322842 consumer
document =  0 score =  164 0.0852028645685 c

IndexError: index out of bounds

In [690]:
wordList = vectorizer.get_feature_names()   
wordList[104:110]
    

['bombs', 'bonusmail', 'break', 'broad', 'business', 'but']

In [691]:
#look at raw documents and then print out all of the non-zer entries in 
#that row of the matrx
bd.data[3]

b'\xef\xbb\xbfHello,\xc2\xa0\n\nI have over 5 years of transcription experience and have been supporting several clients with their transcription requirements. I assure you of quality work and most importantly confidentiality of all the proprietary information / materials. All the work is deleted from our systems as soon as the transcription is completed to the satisfaction of our client.\xc2\xa0\n\nI do sign an NDA if so required. I offer a very low rate of USD 0.60 per audio minute and audios can be transcribed and returned within 24-48 hrs time. What I can also do is a sample transcript and if the sample is to your satisfaction, then we can take it forward from there.\n\nPayment can be made only if you are satisfied with the work and the payment mode could be PayPal, wire transfer or any other means as per your convenience. Let me know what is a good time to call you and discuss this further.\xc2\xa0\n\nLooking forward to hearing from you soon and working for you and once again I as

In [692]:
#Word frequency value for a particular word in a particular document
wordList = vectorizer.get_feature_names()
wordList = [w for w in wordList if not w in ENGLISH_STOP_WORDS]
#len(wordList)#549

In [693]:
for i in range(len(wordList)):
    if(X[3,i] != 0.0):
        print(wordList[i])
    

24
48
60
aid
american
annual
assistant
assure
audio
authorize
begin
best
billions
bills
business
ca
choose
city
contact
continue
date
deadline
department
did
email
fall
fax
giving
hate
having
help
hk
hosting
important
keeping
kyle
launch
love
make
materials
members
mode
modeling
mypoints
november
open
opposite
paid
pay
phone
pieces
possible
privacy
programs
pumpkin
released
reminder
reply
required
richard
room
rooms
sample
seeing
semester
sent
t2x
testing
things
today
unaware
understanding
usd
view
walking
wan
write
year


In [694]:
#idx = wordList.index('fax')
#idx
#finding idx
def search_word(sen):
    words = sen.split()
    #print(words)
    prevword = []
    for w in words:
        if w in wordList:
            prevword.append(w)
    return prevword


In [695]:
def find_idx(wordarray):
    #print(wordarray)
    idxarray = []
    for word in wordarray:
        idx = wordList.index(word)
        idxarray.append(idx)
    return idxarray

In [696]:
def find_class(idxarray):
    class_hs = {} #dictionay of class and probability
    class_score = []
    for j in idxarray:#finding words in all docs
        for i in range(14): 
            #print(j)
            if(X[i,j] != 0.0):
                c_hs = bd.target_names[bd.target[i]]#class for the ith doc
                score = X[i,j]
                t = c_hs,score,j
                #print(c_hs)
                #print(score)
                #class_hs[c_hs] = score
                
                class_score.append(t)
    return class_score
    #if X[i,201] != 0.0:
        #print("fax appears in document", i,"with score",X[i,idx])
#finding index of words

        
        

In [697]:
def ham_prob(hammy):
    ham_idx = []
    for i in hammy:
        ham_idx.append(i[2])
    #print(ham_idx)
    prob = 1
    for i in range(14):    
        for j in range(550):
            if (j not in ham_idx) and (bd.target_names[bd.target[i]] == 'ham') and (X[i,j] != 0.0):
                dif = (1-(X[i,j]))
                #print("dif = ", dif)
                prob *= dif
                
    for i in ham_idx:
        prob *= i
    return prob
                

In [698]:
def spam_prob(spammy):
    spam_idx = []
    for i in spammy:
        spam_idx.append(i[2])
    #print(spam_idx)
    prob = 1
    for i in range(14):    
        for j in range(550):
            if (j not in spam_idx) and (bd.target_names[bd.target[i]] == 'spam') and (X[i,j] != 0.0):
                dif = (1-(X[i,j]))
                #print("dif = ", dif)
                prob *= dif
                
    for i in spam_idx:
        prob *= i
    return prob

In [699]:
word_array = search_word("I would like to work in the web development project")
#word_array

In [700]:
idx_array = find_idx(word_array)
#idx_array

In [701]:
cs_tuple = find_class(idx_array)
#cs_tuple
hammy = []
spammy = []
for i in cs_tuple:
    if (i[0] == 'ham'):
        hammy.append(i)
    #print(hammy)
    else:
        spammy.append(i)
#print(hammy)
        
h_prob = ham_prob(hammy)
s_prob = spam_prob(spammy)

#prior probability for ham is .20 and spam is .80
ham = (h_prob*.20)/((h_prob*.20)+(s_prob*.80)) 
#ham
spam = (s_prob*.80)/((s_prob*.80)+(h_prob*.20))
#spam




In [702]:
print("Ham probability is : %.5f" %ham)
print("Spam probability is : %.5f" %spam)
if(spam > ham):
    print("The email is spam")
else:
    print("The email is ham.")

Ham probability is : 0.99681
Spam probability is : 0.00319
The email is ham.


In [703]:
word_array = search_word("You can view it by clicking here")

In [704]:
idx_array = find_idx(word_array)
#idx_array

In [705]:
cs_tuple = find_class(idx_array)
#cs_tuple
hammy = []
spammy = []
for i in cs_tuple:
    if (i[0] == 'ham'):
        hammy.append(i)
    #print(hammy)
    else:
        spammy.append(i)
#print(hammy)
        
h_prob = ham_prob(hammy)
s_prob = spam_prob(spammy)

#prior probability for both ham is .20 and spam is .80
ham = (h_prob*.20)/((h_prob*.20)+(s_prob*.80)) 
#ham
spam = (s_prob*.80)/((s_prob*.80)+(h_prob*.20))
#spam

In [706]:
print("Ham probability is : %.5f" %ham)
print("Spam probability is : %.5f" %spam)
if(spam > ham):
    print("The email is spam")
else:
    print("The email is ham.")

Ham probability is : 0.00004
Spam probability is : 0.99996
The email is spam


In [ ]:
vectorizer.inverse_transform(X)[3]

In [ ]:
#data_path = os.path.abspath(os.path.join('baise_spamfilter'))
#spam_path = os.path.join(data_path,'spam')
#ham_path = os.path.join(data_path,'ham')    

In [ ]:
#def get_msg(path):
    #with open(path) as con:
        #msg = con.readlines()
        #return ''.join(msg)

In [ ]:
#def get_msgdir(path):
    #filelist = os.listdir(path)
    #filelist = filter(lambda x: x != 'cmds', filelist)
    #all_msgs = [get_msg(os.path.join(path,f)) for f in filelist]
    #return all_msgs

In [ ]:
#all_spam = get_msgdir(spam_path)
#print(all_spam)
#all_ham = get_msgdir(ham_path)

In [ ]:
#bag-of-words counts for the entire SMS corpus
#print('sparse matrix shape:', X.shape)
#print('number of non-seros:', X.nnz)
#print('sparsity: %.2f%%' % (100.0 * X.nnz / 
                            #(X.shape[0] * 
                            #X.shape[1])))
